In [1]:
import pandas as pd
import time as time
time.time()

1671298394.6226265

In [2]:
def read_json_file(path):
    df = pd.DataFrame()
    with pd.read_json(path, lines = True, chunksize =2000000) as df_json:
        for r in df_json:
            df = r
            break;
    return df

In [3]:
reviews_df = read_json_file('yelp_academic_dataset_review.json')
reviews_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [4]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 137.3+ MB


In [5]:
# number of null values
reviews_df.isna().sum().sum()

0

In [6]:
reviews_df = reviews_df[["stars", "text"]]
reviews_df.head()

,stars,text
0,3,"If you decide to eat here, just be aware it is..."
1,5,I've taken a lot of spin classes over the year...
2,3,Family diner. Had the buffet. Eclectic assortm...
3,5,"Wow! Yummy, different, delicious. Our favo..."
4,4,Cute interior and owner (?) gave us tour of up...


In [7]:
# replacing carriage return and new line characters with a space
reviews_df = reviews_df.replace({r'[\r\n]': ' '}, regex=True)
reviews_df.head()

,stars,text
0,3,"If you decide to eat here, just be aware it is..."
1,5,I've taken a lot of spin classes over the year...
2,3,Family diner. Had the buffet. Eclectic assortm...
3,5,"Wow! Yummy, different, delicious. Our favo..."
4,4,Cute interior and owner (?) gave us tour of up...


In [8]:
reviews_df.shape

(2000000, 2)

## Initialize spark

In [9]:
# find spark
import findspark
findspark.init()

# override cloudpickle in order for reading to work
import cloudpickle
import pyspark.serializers
pyspark.serializers.cloudpickle = cloudpickle
import pyspark.sql.functions as f
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.mllib.linalg import Vector

In [10]:
# start session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").config("spark.driver.memory", "15g").appName("YeplReviews").getOrCreate()
sc = spark.sparkContext

In [11]:
# Converting Pandas Dataframe into Spark Dataframe
reviews_df = reviews_df.astype(str) # Converting pandas df to string first
reviews_sdf = spark.createDataFrame(reviews_df)
reviews_sdf.show(5, False)

C:\apps\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\apps\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stars|text                                                                                                                                                       

In [12]:
reviews_sdf.printSchema()

root
 |-- stars: string (nullable = true)
 |-- text: string (nullable = true)



In [13]:
tokenizer = Tokenizer()
tokenizer.setInputCol('text').setOutputCol('words')
reviews_sdf2 = tokenizer.transform(reviews_sdf)
reviews_sdf2.show(5, False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
cv = CountVectorizer()
cvModel = cv.setInputCol("words").setOutputCol("features").fit(reviews_sdf2)
counted = cvModel.transform(reviews_sdf2)
counted.head()

Row(stars='3', text="If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience.   The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", words=['if', 'you', 'decide', 'to', 'eat', 'here,', 'just', 'be', 'aware', 'it', 'is', 'going', 'to', 'take', 'about', '2', 'hours', 'from', 'beginning', 'to', 'end.', 'we', 'have', 'tried', 'it', 'multiple', 'times,', 'because', 'i', 'want', 'to', 'like', 'it!', 'i', 'have', 'been', 'to', "it's", 'other', 'locations', 'in', 'nj', 'and', 'never', 'had', 'a', 'bad', 'experience.', '', '', 'the', 'food', 'is', 'good,', 'but', 'it', 'takes', 'a', 'very', 'long',

In [15]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
schema = StructType([
    StructField("stars", DecimalType(), False),
    StructField("dict", MapType(StringType(),DecimalType()), False)
])

def indices_to_terms(vocabulary):
    def indices_to_terms(xs):
        sv = xs
        mydict = {}
        for i in list(sv.indices):
            print(i)
            mydict[vocabulary[int(i)]] = int(sv[int(i)])
        return mydict
    return udf(indices_to_terms, MapType(StringType(),IntegerType()))

counted.withColumn("result", indices_to_terms(cvModel.vocabulary)("features")).show(5, False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
time.time()

1671298604.1594517